In [1]:
import pandas as pd
import random

names = ['John', 'Emma', 'Michael', 'Olivia', 'William', 'Ava', 'James', 'Isabella', 'Benjamin', 'Mia']

In [27]:
# Get random sales
sales=[]
for _ in range(0,10,1):
    sales.append(random.randint(1000,10000))
col_names=["employee","sales","rank"]
outdf=pd.DataFrame()
outdf["names"]=names
outdf["sales"]=sales
outdf["rank"]=outdf["sales"].rank(method='min', ascending=False).astype(int)
outdf

,names,sales,rank
0,John,8824,2
1,Emma,9829,1
2,Michael,5789,4
3,Olivia,2501,9
4,William,3450,7
5,Ava,4403,6
6,James,5944,3
7,Isabella,1719,10
8,Benjamin,4806,5
9,Mia,2674,8
